Versión renderizada e interactiva: https://nbviewer.jupyter.org/github/marcgror/COVID-19-Data-Analysis/blob/master/COVID.ipynb

# Análisis de los datos del COVID-19 en España
2020 será siempre recordado como el año de la pandemia de COVID-19. España ha sido y es uno de los países más afectados, tanto en la primera ola en marzo como en la siguiente en setiembre.

Para analizar la situación, a nivel nacional, autonómico y provincial, se usarán los datos proporcionados por el repositorio Escovid19data (https://github.com/montera34/escovid19data), que se encarga de recopilar los datos de cada provincia y CCAA, tratarlos y presentarlos de una manera correcta para su visualización y uso.

In [98]:
# Import required packages
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from ipywidgets import widgets
import matplotlib.dates as mdates
import chart_studio.plotly as py
import chart_studio.tools as tls
sns.set_style(style="darkgrid")

In [139]:
# Load each data file from the repository
ccaa = pd.read_csv('ccaa.csv', parse_dates=True)
#provincias = pd.read_csv('https://raw.githubusercontent.com/montera34/escovid19data/master/data/output/covid19-provincias-spain_consolidated.csv')
spain = pd.read_csv('spain.csv', parse_dates=True)

# España 

## Métodos de diagnóstico del COVID-19
En España, el COVID se está detectando principalmente usando las pruebas de PCR y test de anticuerpos, a parte de otras pruebas como el test de antígenos.

La prueba PCR (Polimerasa Chain Reaction) es una prueba de diagnóstico utilizada para detectar fragmentos del material genético de un patógeno. En este caso en concreto, el patógeno se corresponde con el virus SARS-CoV-2. La idea es sencilla: se toma una muestra respiratoria de la persona sospechosa de infección, y se analiza en un laboratorio de microbiología con un PCR en busca de una molécula de ARN del virus. El positivo vendría cuando la prueba detecta ese ARN del virus, mientras será negativo en caso contrario.

Actualmente se dispone tanto de datos de contagiados diagnosticados con PCR como de diagnosticados con otros tipos de prueba, como puede ser el test de anticuerpos o de antígenos.

El siguiente gráfico muestra la contribución en porcentaje de cada tipo de prueba. Como es obvio, más de un 90 % de las detecciones de contagiados son gracias a PCR.

In [168]:
labels = ['PCR', 'Test AC', 'Otras', 'Desconocida']
values = list(spain[['num_casos_prueba_pcr', 'num_casos_prueba_test_ac', 'num_casos_prueba_otras', 'num_casos_prueba_desconocida']].sum())
fig = go.Figure(data=[go.Pie(labels=labels, values=values)], layout=dict(title='Contribución de cada método de detección'))
fig.show()

Mientras que durante el mes de octubre encontramos una distribución de pruebas similar, con los test PCR dominando por mucho y los test de anticuerpos en segundo lugar:

In [170]:
fig = go.Figure(layout=dict(title='Contribución de cada método de detección en el último mes'))
fig.add_trace(go.Bar(x=spain['fecha'], y=spain['num_casos_prueba_pcr'], name='PCR'))
fig.add_trace(go.Bar(x=spain['fecha'], y=spain['num_casos_prueba_test_ac'], name='Test AC'))
fig.add_trace(go.Bar(x=spain['fecha'], y=spain['num_casos_prueba_otras'], name='Otras'))
fig.add_trace(go.Bar(x=spain['fecha'], y=spain['num_casos_prueba_desconocida'], name='Desconocida'))
fig.update_layout(barmode='stack', xaxis={'categoryorder': 'category ascending'}, width=1000)
fig.update_xaxes(range=('2020-10-1', '2020-10-31'))
fig.show()

## Evolución de los contagios
En la siguiente tabla se muestran los casos diarios, acumulados, el incremento diario, en los últimos 7 y 14 días, la media de casos en los últimos 7 y 14 días, la incidéncia acumulada en los últimos 7 y 14 días y la acumulada para los últimos 10 días a nivel nacional:

In [210]:
fig = go.Figure(data=[go.Table(
    columnwidth=[100, 90, 90, 180, 100, 120, 140, 140, 100, 100, 100],
    header=dict(values=list(['Fecha', 'Casos nuevos', 'Casos totales', 'Incremento de casos', 'Casos 7 días', 'Casos 14 días', 'Media casos 7 días', 'Media casos 14 días', 'IA 7 días', 'IA 14 días', 'IA total']),
                fill_color='rgb(124,124,124)',
                align='center', font=dict(color='white')),
    cells=dict(values=spain[['fecha', 'num_casos', 'cases_accumulated', 'cases_inc', 'cases_7_days', 'cases_14_days', 'avg_cases_7_days', 'avg_cases_14_days', 'ia_100000_week', 'ia_100000_2week', 'ia_accumulated']].tail(10).transpose().values.tolist(),
               fill_color='rgb(179, 179, 179)',
               align='center', height=25))
])
fig.update_layout(width=1200, height=500, title='Datos de contagios para los últimos 10 días')
fig.show()

La curva de contagiados en España desde el inicio de la pandémia se muestra a continuación:

In [183]:
fig = go.Figure()
#fig.add_trace(go.Bar(x=spain['fecha'], y=spain['num_casos_prueba_pcr'], name= 'Nuevos PCR'))
fig.add_trace(go.Bar(x=spain['fecha'], y=spain['num_casos'], name='Nuevos casos'))
fig.update_layout(legend=dict(orientation='h', bgcolor='LightSteelBlue'), width=1200, title='Casos diarios desde el inicio de la pandémia')
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=22000, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=19000, showarrow=False)
#fig.add_vrect(x0='2020-10-25', x1='2021-04-24', y1=20000, line_width=0, fillcolor='grey', opacity=0.2)
#fig.add_annotation(text='Segundo Estado Alarma', hovertext='Periodo de declaración del segundo Estado de Alarma de la pandémia', x='2020-10-24', y='19000', showarrow=False)
fig.show()

Las dos olas se pueden diferenciar claramente. Los efectos del confinamiento, desescalada y fin del Estado de Alarma son también evidentes, ya que el número de contagios diarios decae a mínimos practicamente hasta principios de julio, fruto del parón social y laboral del país. Es entonces cuando empiezan a aumentar los nuevos positivos, seguramente porque la "Nueva Normalidad" lleva vigente 1-2 semanas en toda España, y por lo tanto la población empieza poco a poco a interactuar de nuevo. El hecho de que esta vuelta a la socialización y al trabajo se diera en verano seguramente no ayuda tampoco.

Si nos fijamos en los positivos acumulados, las tendencias son prácticamente las mismas, como es lógico:

In [184]:
fig = go.Figure(data=[go.Bar(x=spain['fecha'], y=spain['cases_accumulated'])], layout=go.Layout(title=go.layout.Title(text='Casos PCR acumulados desde el inicio de la pandémia')))
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=1400000, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=1200000, showarrow=False)
fig.update_layout(width=1200)
fig.show()

In [125]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
fig.add_trace(go.Bar(x=spain['fecha'], y=spain['hospitalized'], name='Hospitalizados'), row=1, col=1)
fig.add_trace(go.Bar(x=spain['fecha'], y=spain['intensive_care'], name='Cuidados intensivos'), row=2, col=1)
fig.add_trace(go.Bar(x=spain['fecha'], y=spain['recovered'], name='Recuperados'), row=3, col=1)
fig.update_layout(legend=dict(orientation='h', bgcolor='LightSteelBlue'), title='Hospitalizados, en UCI y recuperados')
fig.show()

KeyError: 'hospitalized'

Otros indicadores útiles para valorar el estado actual de la pandemia son el número de positivos hospitalizados, en la UCI y recuperados. Como se puede observar, en el peor momento de la primera ola (finales de marzo-mayo) es cuando se produjo el mayor número de personas hospitalizadas o en la UCI, mientras que los recuperados iban aumentando a medida que pasaba las semanas. Durante el inicio del verano los ingresados disminuyen, fruto del descenso de los contagios debido al confinamiento y la desescalada. Una vez llega Agosto, se empiezan a notar los efectos de la "nueva normalidad", y los hospitalizados e ingresados en UCI por COVID empiezan a aumentar de nuevo.

Durante todo este tiempo, las cifras de recuperados no hacen más que aumentar, debido seguramente al gran número de personas infectadas con anterioridad.

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10,5), sharex=True)
sns.lineplot(x='date', y='hospitalized', data=spain, label='Hospitalizados', ax=ax1)
ax1.axvline(x=pd.to_datetime('21-06-2020'), color='r')
ax1.text(x=pd.to_datetime('22-06-2020'), y=10000,  rotation=90, s='F.E.A.', color='r', bbox=dict(facecolor='black', alpha=0.20))
sns.lineplot(x='date', y='intensive_care', data=spain, label='Cuidados intensivos', ax=ax2)
ax2.axvline(x=pd.to_datetime('21-06-2020'), color='r')
ax2.text(x=pd.to_datetime('22-06-2020'), y=1000,  rotation=90, s='F.E.A.', color='r', bbox=dict(facecolor='black', alpha=0.20))
sns.lineplot(x='date', y='recovered', data=spain, label='Recuperados', ax=ax3)
ax3.axvline(x=pd.to_datetime('21-06-2020'), color='r',)
ax3.text(x=pd.to_datetime('22-06-2020'), y=78000,  rotation=90, s='F.E.A.', color='r', bbox=dict(facecolor='black', alpha=0.20))
f.autofmt_xdate()
plt.legend()

### Incidéncia acumulada
La incidéncia acumulada (IA) es una forma de medir el impacto de la pandémia en cada territorio teniendo en cuenta su población. Por lo tanto, permite comparar valores entre distintas regiones/paises y establecer valores máximos que, una vez superados, exijan aplicar medidas de restricción de movilidad.

La evolución de la IA en 7 y 14 días durante toda la pandémia se muestra a continuación:

In [189]:
fig = go.Figure(data=[go.Scatter(x=spain['fecha'], y=spain['ia_100000_2week'], mode='lines', name='IA 14 días')], layout=go.Layout(title=go.layout.Title(text='Incidencia acumulada')))
fig.add_trace(go.Scatter(x=spain['fecha'], y=spain['ia_100000_week'], mode='lines', name='IA 7 días'))
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=580, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=530, showarrow=False)
fig.update_layout(width=1200, legend=dict(orientation='h', bgcolor='LightSteelBlue'))
fig.show()

Se observa una tendéncia similar a la curva de contágios.

## Fallecidos

El siguiente paso es el número de fallecidos. A continuación se muestra la evolución del número de fallecidos diario diagnosticados con COVID desde el inicio de la pandémia:

In [202]:
fig = go.Figure(data=[go.Bar(x=spain['fecha'], y=spain['deceased'])], layout=go.Layout(title=go.layout.Title(text='Fallecimientos diarios desde el inicio de la pandémia')))
fig.update_yaxes(range=[0,1000])
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=1000, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=900, showarrow=False)
fig.update_layout(width=1200)
fig.show()

Durante la primera ola, se alcanzaron cifras diarias de más de 1000 muertos diarios positivos por COVID. Gracias al confinamiento, estas cifras fueron dismuyendo lentamente durante mayo y junio, llegando a su mínimo durante el verano. Una vez los contagios e infectados vuelven a incrementarse a finales de agosto, los fallecidos empiezan a aumentar también.

In [199]:
fig = go.Figure(data=[go.Bar(x=spain['fecha'], y=spain['deceased_accumulated'])], layout=go.Layout(title=go.layout.Title(text='Fallecimientos acumulados desde el inicio de la pandémia')))
fig.add_vrect(x0='2020-03-14', x1='2020-06-21', y1=40000, line_width=0, fillcolor='grey', opacity=0.2)
fig.add_annotation(text='Primer Estado Alarma', hovertext='Periodo de declaración del primer Estado de Alarma de la pandémia', x='2020-04-24', y=35000, showarrow=False)
fig.update_layout(width=1200)
fig.show()

En el acumulado, se puede observar un estancamiento en el número de fallecidos durante el verano, pero a partir de los últimos días de agosto se observa de nuevo una tendencia al aumento.

## CCAA

A partir de los datos separados por Comunidades Autónomas es posible seguir la evolución de la pandemia en estas.

In [ ]:
ccaa.info(verbose=True)

Empecemos con una tabla resumen: en ella se muestran los casos acumulados diagnosticados por PCR, el número de nuevos casos (diarios, en 7 y en 14 días) y la IA por 100000 habitantes para cada comunidad autónoma, ordenado por IA:

In [142]:
fig = go.Figure(data=[go.Table(
    columnwidth=[100, 90, 90, 180, 100, 120, 140, 140, 100, 100, 100],
    header=dict(values=list(['Fecha', 'CCAA', 'Casos totales', 'Casos diarios', 'Casos 7 días', 'Casos 14 días', 'IA 7 días']),
                fill_color='white',
                align='center'),
    cells=dict(values=ccaa.loc[ccaa['fecha']=='2020-10-25',['fecha', 'ccaa','cases_accumulated_PCR', 'num_casos', 'cases_7_days', 'cases_14_days', 'ia_100000_week']].sort_values(by='ia_100000_week', ascending=False).transpose().values.tolist(),
               fill_color='rgb(107, 174, 214)',
               align='center'))
])
fig.update_layout(width=1200, height=800, paper_bgcolor="LightSteelBlue")
fig.show()

En la siguiente tabla se muestran los fallecimientos acumulados, diarios, incrementos respecto al día anterior, en la última semana y por cada 100000 habitantes, ordenados por este último:

In [160]:

fig = go.Figure(data=[go.Table(
    columnwidth=[50, 90, 70, 70, 70, 70, 120],
    header=dict(values=list(['Fecha', 'CCAA', 'Fallecidos totales', 'Fallecidos diarios', 'Incremento de fallecidos', 'Fallecidos 7 días', 'Fallecidos por 100000 habitantes ']),
                fill_color='white',
                align='center'),
    cells=dict(values=ccaa.loc[ccaa['fecha']=='2020-10-25',['fecha', 'ccaa', 'deceased_accumulated', 'deceased', 'deceased_inc', 'deaths_7_days', 'deceased_per_100000']].sort_values(by='deceased_per_100000', ascending=False).transpose().values.tolist(),
               fill_color='rgb(107, 174, 214)',
               align='center'))
])
fig.update_layout(width=1200, height=800, paper_bgcolor="LightSteelBlue")
fig.show()


Aunque una imagen vale más que una tabla. En la siguiente figura se muestra el número de casos diagnosticados por PCR para cada autonomia:

In [98]:
fig = go.Figure(layout=go.Layout(title=go.layout.Title(text='Casos de PCR acumulados')))
fig.add_trace(go.Bar(y=ccaa['ccaa'], x=ccaa['cases_accumulated'], orientation='h'), )
fig.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'}, height=600, width=800)
fig.show()

In [161]:
def create_interactive_plot(df, ccaa_series, dates, traces, title_text):
    widget = widgets.Dropdown(
    options=ccaa_series.unique().tolist(),
    description='CCAA'
    )
    g = go.FigureWidget(layout=go.Layout(
                        title=dict(
                            text=title_text
                        ), legend=dict(orientation='h', bgcolor='LightSteelBlue'),
                        barmode='overlay'
                    ))
    for trace, trace_name in traces:
        g.add_trace(go.Bar(x=dates, y=df[trace], name=trace_name))

    def validate():
        if widget.value in ccaa_series.unique():
            return True
        else:
            return False

    def response(change):
        if validate():
            filter_list = [i for i in ccaa_series == widget.value]
            temp_df = df[filter_list]
            x = temp_df['fecha']
            i=0
        with g.batch_update():
            for trace, trace_name in traces:
                g.data[i].x = x
                g.data[i].y = temp_df[trace]
                i+=1
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = ''
            g.layout.yaxis.title = ''
    widget.observe(response, names='value')
    return widgets.VBox([widget, g])

In [162]:
create_interactive_plot(ccaa, ccaa['ccaa'], ccaa['fecha'], [['num_casos', '# casos'], ['num_casos_prueba_pcr', '# casos PCR']], 'Casos PCR por CCAA')

In [163]:
create_interactive_plot(ccaa, ccaa['ccaa'], ccaa['fecha'], [['activos', 'Activos'], ['hospitalized', 'Hospitalizados'], ['daily_deaths', 'Fallecidos']], 'Casos activos, hospitalizados y fallecidos por CCAA' )

KeyError: 'activos'

## Provincias

In [ ]:
create_interactive_plot(provincias, provincias['province'], provincias['date'], [['num_casos', '# casos'], ['num_casos_prueba_pcr', '# casos PCR']], 'Casos PCR por provincia')